In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm

In [2]:
HLA_df = pd.read_csv("DGN_HLA_df.csv")
vdj_table = pd.read_csv("DGN_vdj_usages_table.csv")
cov_df = pd.read_csv("DGN_covariates_df.csv")
query = "TCR"
cov_df["lane"] = cov_df["fcid"] + "_" + cov_df["lane"].astype("str") # covariates

### Do any cohort filtering before analysis

In [3]:
# plt.hist(HLA_df['HLA_DQB1_0301'], bins=50)

# DQA1_columns = [x for x in HLA_df.columns if x.startswith("HLA_DQA1")]

# for col in DQA1_columns:
#     fig, ax = plt.subplots()
#     plt.title(col)
#     plt.hist(HLA_df[HLA_df['HLA_DQB1_0301']>0.5][col], bins=30)

In [4]:
# HLA_df = HLA_df[HLA_df['HLA_DQA1_0501']>0.5] # only look at certain DQA1 groups

## setting up all stats stets below

In [5]:
pat_info_df = pd.merge(cov_df, HLA_df, on="patid", how="inner")
all_df = pd.merge(pat_info_df, vdj_table, on="patid", how="inner")
all_df = pd.get_dummies(all_df, columns=['fcid', 'lane'])
tcr_columns = [x for x in all_df.columns if x.startswith("TR")]
hla_columns = [x for x in all_df.columns if x.startswith("HLA")]
fcid_columns = [x for x in all_df.columns if x.startswith("fcid")]
lane_columns = [x for x in all_df.columns if x.startswith("lane")]

## OLS

In [6]:
group_0602 = "all" # neg, pos, all

In [7]:
ols_df = pd.DataFrame()
ols_df[["patid"]] = all_df[["patid"]]
ols_df['HLA_DQB1_0301'] = (all_df['HLA_DQB1_0301']>0.5).astype("int")
ols_df['HLA_DQB1_0602'] = (all_df['HLA_DQB1_0602']<0.5).astype("int")
ols_df[tcr_columns] = all_df[tcr_columns]
ols_df[fcid_columns] = all_df[fcid_columns]
ols_df[lane_columns] = all_df[lane_columns]

In [8]:
if group_0602 == "pos":
    ols_df = ols_df[ols_df['HLA_DQB1_0602'] == 1]
elif group_0602 == "neg":
    ols_df = ols_df[ols_df['HLA_DQB1_0602'] == 0]

In [9]:
# FOUR SEPARATE GROUPS

# ols_df['+0602+0301'] = ((ols_df['HLA_DQB1_0602'] == 1) & (ols_df['HLA_DQB1_0301'] == 1)).astype("int")
# ols_df['+0602-0301'] = ((ols_df['HLA_DQB1_0602'] == 1) & (ols_df['HLA_DQB1_0301'] == 0)).astype("int")
# ols_df['-0602+0301'] = ((ols_df['HLA_DQB1_0602'] == 0) & (ols_df['HLA_DQB1_0301'] == 1)).astype("int")
# ols_df['-0602-0301'] = ((ols_df['HLA_DQB1_0602'] == 0) & (ols_df['HLA_DQB1_0301'] == 0)).astype("int")
# #ols_df['HLA_DQB1_0602*0301'] = ols_df['HLA_DQB1_0301']*ols_df['HLA_DQB1_0602']
# np.sum(ols_df['+0602+0301'].values), np.sum(ols_df['+0602-0301'].values), np.sum(ols_df['-0602+0301'].values), np.sum(ols_df['-0602-0301'].values)

In [10]:
print("doing {} tests".format(len(tcr_columns)))
x_columns = ['HLA_DQB1_0301'] + fcid_columns#lane_columns}  #['+0602+0301', '+0602-0301', '-0602+0301', '-0602-0301']
res_df = pd.DataFrame()
for tcr in tcr_columns:
    y_column = tcr
    X = ols_df[x_columns]
    y = ols_df[y_column]
    X = sm.add_constant(X) # constant is always added
    mod = sm.OLS(y, X)
    res = mod.fit()
    
    tcr_df = pd.DataFrame()
    tcr_df['coef'] = res.params
    tcr_df['se'] = res.bse
    tcr_df['pvalue'] = res.pvalues
    tcr_df['tvalue'] = res.tvalues
    res_df[tcr] = tcr_df.loc['HLA_DQB1_0301']
    
res_df = res_df.transpose()
res_df 

doing 157 tests


/home/ashteng/36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,coef,se,pvalue,tvalue
TRAJ1,0.000081,0.000135,0.547685,0.601480
TRAJ10,0.000171,0.000513,0.739422,0.332730
TRAJ11,0.000212,0.000322,0.509895,0.659286
TRAJ12,-0.000372,0.000405,0.358894,-0.917984
TRAJ13,0.001905,0.000544,0.000488,3.501275
...,...,...,...,...
TRBV7-6,-0.000126,0.000363,0.728078,-0.347801
TRBV7-7,0.000175,0.000123,0.155695,1.420993
TRBV7-8,0.000337,0.000471,0.474919,0.714824
TRBV7-9,0.002255,0.000940,0.016688,2.398380


In [11]:
res_df = res_df.sort_values("pvalue", ascending=True)

In [12]:
res_df = res_df.reset_index().rename(columns={"index":"TCR"})
res_df.to_csv("dgn_res_df_0602{}.csv".format(group_0602), index=None)

In [ ]:
# print("doing {} tests".format(len(tcr_columns)))
# pvalue_mat = []
# coefs_mat = []
# x_columns = ['HLA_DQB1_0301'] + fcid_columns#lane_columns}  #['+0602+0301', '+0602-0301', '-0602+0301', '-0602-0301']
# for tcr in tcr_columns:
#     y_column = tcr
#     X = ols_df[x_columns]
#     y = ols_df[y_column]
#     X = sm.add_constant(X) # constant is always added
#     mod = sm.OLS(y, X)
#     res = mod.fit()
#     #print('Parameters:\n', res.params)
#     #print('R2: ', res.rsquared)
#     #print(tcr, "{:.4f}".format(res.params[]), "{:.4f}".format(res.rsquared), "{:.4f}".format(res.pvalues[tcr]))
#     coefs = [res.params[x] for x in x_columns]
#     coefs_mat.append(coefs)
#     #print(res.summary())
#     pvalues = [res.pvalues[x] for x in x_columns]
#     pvalue_mat.append(pvalues)

# pvalue_mat = np.array(pvalue_mat)
# neglog_pvalue_mat = -np.log(np.array(pvalue_mat))
# coefs_mat = np.array(coefs_mat)

# # visualize all covariates
# fig, ax = plt.subplots(figsize=(20,10))
# plt.imshow(neglog_pvalue_mat)
# plt.colorbar()
# plt.title("DGN -log(pvalues). first column is 0301, the rest are flow cell ids")

#-np.log(0.05/len(tcr_columns))

# fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=((pvalue_mat.shape[1]//2+3)*2, (pvalue_mat.shape[0]//2)*2))
# fig.tight_layout(pad=3.0)
# ax1 = sns.heatmap(pvalue_mat, cmap='seismic', center=0, vmin=0, vmax=10, cbar_kws={'label': "-log(pvalue)"}, ax=ax1)
# ax2 = sns.heatmap(coefs_mat, cmap='seismic', center=0, vmin=-0.005, vmax=0.005, cbar_kws={'label': "coef"}, ax=ax2)
# #plt.ylabel(X_name)
# #plt.xlabel(Y_name)
# #plt.yticks(rotation=0)
# #plt.xticks(rotation=90)

# fig.suptitle("[ONLY 0602 POSITIVE PEOPLE] 0301 predicting {} expression (DGN)".format(query), y=1)

# ax1.set_title("-log(p-values)")
# ax1.set_yticklabels(['bias'] + tcr_columns, rotation=0)
# ax1.set_xticklabels(x_columns, rotation=90)
# bottom, top = ax1.get_ylim()
# #ax1.set_ylim(bottom + 0.5, top - 0.5)

# ax2.set_title("OLS coefficients")
# ax2.set_yticklabels(tcr_columns, rotation=0)
# ax2.set_xticklabels(x_columns, rotation=90)
# bottom, top = ax2.get_ylim()
# ax2.set_ylim(bottom + 0.5, top - 0.5)

## GLMNET

In [ ]:
expr_mat.shape, HLA_mat.shape

In [ ]:
# Import relevant modules and setup for calling glmnet
%matplotlib inline
import glmnet_python
from glmnet import glmnet
import sys
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
from glmnet import glmnet; from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict

In [ ]:
DQB1_mask = np.array([1 if name.startswith("HLA_DQB1") else 0 for name in HLA_names])
DQB1_mask = np.argwhere(DQB1_mask==1).flatten()

In [ ]:
DQB1_mask

In [ ]:
X_matrix = expr_mat
Y_matrix = HLA_mat[:,DQB1_mask]
X_name = "{} gene expression".format(query)
Y_name = "HLA DQB1 haplotypes"
X_labels = list(expr_names)
Y_labels = list(HLA_names[DQB1_mask])

In [ ]:
for lambda_pick_strategy in ['lambda_1se', 'lambda_min']:
    
    coefs = []
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('default')
    #print("all TCR functional genes")
    #print("Response Variable, %Deviation Explained by Model")
    for t in range(Y_matrix.shape[1]): # for each HLA subtype, use it as response variable
        y = Y_matrix[:,t]
        non_nan_idx = np.argwhere(1-np.isnan(y)).flatten()
        y = y[non_nan_idx]
        X = X_matrix[non_nan_idx]
        cvfit = cvglmnet(x = X, y = y, ptype = 'mse')
        coefs_i = cvglmnetCoef(cvfit, s = lambda_pick_strategy)
        #fit = glmnet(x = X, y = y, family = 'gaussian', alpha = 1.0, nlambda = 20)
        #print("{}, {}".format(Y_labels[t], fit['dev'][cv_chosen_lambda])
        #coefs_i = glmnetCoef(fit, s = scipy.float64(cv_chosen_lambda), exact = False)
        coefs.append(coefs_i)
    print(cvfit['lambda_1se'], cvfit['lambda_min'])

    coefs = np.hstack(coefs)
    fig, ax = plt.subplots(figsize=(coefs.shape[1]//2, coefs.shape[0]//2))
    ax = sns.heatmap(coefs, cmap='seismic', center=0, cbar_kws={'label': "Regression Coefficient"})
    plt.ylabel(X_name)
    plt.xlabel(Y_name)
    plt.yticks(rotation=0)
    plt.xticks(rotation=90)
    ax.set_yticklabels(['bias'] + X_labels)
    ax.set_xticklabels(Y_labels)
    plt.title("Lasso Regression coefficients ({}) of {} predicting {}".format(lambda_pick_strategy, X_name, Y_name))
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
# import pickle
# import os

# if os.path.exists("gaussian_coefs.pkl"):
#     coefs = pickle.load(open("gaussian_coefs.pkl", "rb"))
# else:
#     pickle.dump(coefs, open("gaussian_coefs.pkl", "wb"))

In [ ]:
# np.mean(coefs.flatten())
# _ = plt.hist(coefs.flatten(), bins=50)

## T test

In [ ]:
import scipy
import statsmodels.stats.multitest as multitest
import statsmodels.api as sm

In [ ]:
for tcr in tcr_columns:
    test_0301 = test_df[test_df["HLA_DQB1_0602"] > 0][tcr].values
    test_other = test_df[test_df["HLA_DQB1_0602"] <= 0][tcr].values
    tstat, pvalue = scipy.stats.ttest_ind(test_0301, test_other)
    print(tcr, np.mean(test_0301)-np.mean(test_other), tstat, pvalue)
    if pvalue < 0.05:
        print("^")

## Correlation

In [ ]:
hla_arr = dqb_0602_usage["HLA_DQB1_0301"].values
print("doing {} tests".format(len(tcr_columns)))

corrcoefs = []
pvalues = []
   
for tcr in tcr_columns:
    exp_arr = dqb_0602_usage[tcr].values
    corrcoef, pvalue = scipy.stats.pearsonr(hla_arr, exp_arr)
    corrcoefs.append(corrcoef)
    pvalues.append(pvalue)

# get 5% FDR corrected p-values
rejected, corrected_pvalues = multitest.fdrcorrection(pvalues)

In [ ]:
print("TCR, corrcoef, corrected_pvalue, is_rejected")
for i in range(len(tcr_columns)):
    print(tcr_columns[i], corrcoefs[i], corrected_pvalues[i], rejected[i])
    if rejected[i] == True:
        print("*")

In [ ]:
import seaborn as sns

In [ ]:
for tcr in tcr_names:
    fig, ax = plt.subplots()
    sns.distplot(dqb_0602_usage[dqb_0602_usage['HLA_DQB1_0301']>0.5][tcr], kde=False, label="0602+0301")
    sns.distplot(dqb_0602_usage[dqb_0602_usage['HLA_DQB1_0301']<=0.5][tcr], kde=False, label="0602+other")
    
    plt.title(tcr)
    plt.xlabel("usage")
    plt.legend()

In [ ]:
sns.distplot(all_df[all_df['HLA_DQB1_0301']>0.5]['TRAV13-1'], bins=30, kde=False, label="0301")
sns.distplot(all_df[all_df['HLA_DQB1_0301']<=0.5]['TRAV13-1'], bins=30, kde=False, label="non-0301")

plt.xlabel("TRAV13-1 usage")
plt.legend()

In [ ]:
len(all_df[all_df['HLA_DQB1_0301']<=0.5])